In [227]:
import requests
from bs4 import BeautifulSoup, SoupStrainer 
import sys
import string
import os
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize 
from sklearn.feature_extraction.text import CountVectorizer
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.cross_validation import train_test_split

In [10]:
url = 'http://homeopathicremediesandtreatment.com/'

In [8]:
print len('http://')#homeopathicremediesandtreatment.com/')

7


In [37]:
#Function takes website request text and scrapes all links on that site
def Scrape_Whole_Site(url):
    link_list = []
    master_list = []
    successes = 0
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    master_links = soup.find_all('a')
    
    for link in master_links:
        link_list.append(link['href'])
        master_list.append(link['href'])
    
    #for link in link_list:
        #print link
        #print link[:len(url)]
    
    for link in link_list:
        try:
            sub_request = requests.get(link)
        except:
            pass
        sub_soup = BeautifulSoup(sub_request.text, 'html.parser')
        sub_links = sub_soup.find_all('a')
        for sub_link in sub_links:
            try:
                if sub_link['href'][:len(url)] == url:
                    if sub_link['href'] not in master_list:
                        #print sub_link['href']
                        master_list.append(sub_link['href'])
                        successes += 1
                        sys.stdout.write("\r" + 'Sucesses: ' + str(successes))   
            except:
                pass
        
    first_len = len(master_list)
    set_list = set(master_list)
    set_len = len(master_list)
    
    return first_len, set_len
    
first_len, set_len = Scrape_Whole_Site(url)
print ' '
print 'Length: ' + str(first_len)
print 'Set len: ' + str(set_len)
print 'Done'
for url in master_list:
    print url

Sucesses: 1682 
Length: 1724
Set len: 1724
Done
http://homeopathicremediesandtreatment.com/Homeopathic-Remedies-and-Treatment.php
http://homeopathicremediesandtreatment.com/Hahnemann-Homeopathy.php
http://homeopathicremediesandtreatment.com/Gemmotherapy-What-is-Gemmotherapy.php
http://homeopathicremediesandtreatment.com/Oligotherapy-Oligo-Trace-Elements.php
http://homeopathicremediesandtreatment.com/Biochemic-Tissue-Salts-Schuessler-Cell-Salts.php
http://homeopathicremediesandtreatment.com/Organotherapy-What-is-Organotherapy.php
http://homeopathicremediesandtreatment.com/Liver-Detoxification--Homeopathy.php
http://homeopathicremediesandtreatment.com/Eczema-Skin-Eruption-Suppression.php
http://homeopathicremediesandtreatment.com/Homeopathic-Case-Taking-Questionnaire-Homeopathy.php
http://homeopathicremediesandtreatment.com/Acid-base-Balance-Body-pH-Balance.php
http://homeopathicremediesandtreatment.com/10-Food-Combination-Rules.php
http://homeopathicremediesandtreatment.com/Arthritis-Ca

In [41]:
letters = [string.ascii_lowercase]
letters

['abcdefghijklmnopqrstuvwxyz']

In [48]:
#http://www.homeopathycenter.org/   #doesn't take many requests to jam up server
url = 'http://www.homeopathycenter.org/find-remedy/a'# + letters[0]
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
text = soup.find_all('p')
soup.text

ConnectionError: HTTPConnectionPool(host='www.homeopathycenter.org', port=80): Max retries exceeded with url: /find-remedy/a (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x1081cb690>: Failed to establish a new connection: [Errno 60] Operation timed out',))

###Assumed kook source

In [154]:
#http://www.homeoint.org    #Scrapes Homeopathy site
for i in range(40):
    url = 'http://www.homeoint.org/hompath/articles/' + str(i) + '.html'
    r = requests.get(url)
    if r.status_code != 404:
        try:
            soup = BeautifulSoup(r.text, 'html.parser')
            titles = soup.find_all('title')
            the_title = titles[0].text
            text = soup.find_all('br')
            content = text[0].text
            with open('/users/markregalla/desktop/metis/Project4/HomeoArticles/page_' 
                      + str(i) +'.txt', 'w') as myfile:
                #myfile.write((the_title + '\n' + content).encode("UTF-8"))
                myfile.write((the_title + content).encode("UTF-8"))
            myfile.close
        except:
            pass
    sys.stdout.write("\r" + str(i))   

39

###Assumed genuine source

In [135]:
#Mayo Clinic Drugs and supplements articles
url = 'http://www.mayoclinic.org/drugs-supplements'
r = requests.get(url)
print r.status_code
soup = BeautifulSoup(r.text, 'html.parser')
i = 0
ref_list = []
for link in soup.find_all('a'):
    if i in range(108, 149):
        ref_list.append(link['href'])
    i += 1
k = 0
for link in ref_list:
    r = requests.get('http://www.mayoclinic.org/' + link)
    soup = BeautifulSoup(r.text, 'html.parser')
    texts = soup.find_all('p')
    text_list = []
    i = 0 
    for text in texts:
        text_list.append(text.text)
        i += 1
    
    document = ''
    for element in text_list[3:len(texts) - 8]:
        document = document + str(element.encode("UTF-8"))
    try:
        with open('/users/markregalla/desktop/metis/Project4/MayoArticles/page_' 
                          + str(k) +'.txt', 'w') as myfile:
            myfile.write(document.encode("UTF-8"))
        myfile.close
    except:
        pass
    sys.stdout.write("\r" + str(k)) 
    k += 1

200
40

###Run scraped articles through Naive Bayes classifier

In [141]:
#Mayo list
Mayo_list = []
for article in os.listdir('/users/markregalla/desktop/metis/Project4/MayoArticles'):
    with open("/users/markregalla/desktop/metis/Project4/MayoArticles/" 
              + article, "r") as myfile:
        stuff = myfile.read()
    myfile.close
    if stuff != '':
        Mayo_list.append(stuff)
print len(Mayo_list)
Mayo_list

35


['Lactobacillus acidophilus is a member of the Lactobacillus genus of bacteria. These bacteria can be found in the mouth, intestine, and vagina. L. acidophilus is thought to benefit health, since it produces vitamin K and lactase. However, L. acidophilus cannot make many other vitamins and amino acids. Because of this, L. acidophilus is mostly found in the upper gastrointestinal (GI) tract where there are higher amounts of these nutrients.L. acidophilus is commonly used in food, such as yogurt, other dairy products, and fermented soy products, such as miso and tempeh.L. acidophilus is one of the most commonly used probiotics, microorganisms that are used to promote health. Probiotics are different from prebiotics, which are complex sugars that may support the growth of "good" bacteria in the intestinal tract. The word "synbiotic" means that a product has both a probiotic and a prebiotic.There is good evidence for the use of L. acidophilus in treating vaginal infections. However, there 

In [184]:
#Homeopathic list
Homeo_list = []
k = 0
for article in os.listdir('/users/markregalla/desktop/metis/Project4/HomeoArticles'):
    with open('/users/markregalla/desktop/metis/Project4/HomeoArticles/' 
              + article) as myfile:
        stuff = myfile.read()
    myfile.close
    if stuff != '':
        Homeo_list.append(stuff.decode("ascii", "ignore"))
    k += 1
    if k == 35:
        break
print Homeo_list
'''
total_list = []
for word in Homeo_list[0]:
    total_list.append(word.decode("ascii", "ignore"))
print total_list[0]
'''

[u'Tresorie - Application of homoeopathic Principles in of Mental States\n[Application Of Homoeopathic Principles In The Treatment Of Mental States]\n Middletown, N.Y.\n      \n THE object of this paper is to reveal\n      and evaluate homoeopathic principles in the treatment of mental reactions.\n      Such an approach is being practiced at the Middletown State homoeopathic\n      Hospital where our observations were made. In this paper we will also\n      bring forth the observations and conclusions arrived at by others as to\n      the effects of certain homoeopathic remedies in the treatment of certain\n      mental states. As Dr. Talcott once wrote, "The means used for the\n      recovery of patients are numerous and varied. We cannot pin our faith to a\n      single measure. We week to adopt all which experience has proved likely to\n      be beneficial." It will be out chief aim to show just what\n      homoeopathic remedies are most effective in mental disease, the principles\n

'\ntotal_list = []\nfor word in Homeo_list[0]:\n    total_list.append(word.decode("ascii", "ignore"))\nprint total_list[0]\n'

In [213]:
for word in Mayo_list:
    Mayo_word_token = word_tokenize(word)
    Mayo_sent_token = sent_tokenize(word)
for word in Homeo_list:
    Homeo_word_token = word_tokenize(word)
    Homeo_sent_token = sent_tokenize(word)
#Homeo_token

In [264]:
%matplotlib inline
vectorizer = CountVectorizer(ngram_range=(1,2))
vectorizer.fit(Mayo_word_token)
vectorizer.fit(Homeo_word_token)

#difference between 2 below?
#none after comparing both
#x = vectorizer.transform(Mayo_word_token)
#mayo_train_vector = vectorizer.fit_transform(Mayo_word_token)

#homeo_train_vector = vectorizer.fit_transform(Homeo_word_token)
mayo_train_vector = vectorizer.transform(Mayo_word_token)
homeo_train_vector = vectorizer.transform(Homeo_word_token)

mayo_train_y = [1 for x in range(len(Mayo_word_token))]
homeo_train_y = [0 for x in range(len(Homeo_word_token))]
clf = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
clf.fit(mayo_train_vector, mayo_train_y)
clf.fit(homeo_train_vector, homeo_train_y)
print clf.predict(vectorizer.transform(['aloe']))

#print vectorizer.get_feature_names()
x_back = mayo_train_vector.toarray()
#df = pd.DataFrame(x_back, columns=vectorizer.get_feature_names())
#df.transpose()
#plt.scatter(df)

[0]


In [229]:
X = Mayo_word_token
y = Homeo_word_token
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print X_train, X_test, y_train, y_test

ValueError: Found arrays with inconsistent numbers of samples: [ 136 3100]

In [ ]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))


classes = np.array(Mayo_sent_token)
model = MultinomialNB().fit(doc_vectors, classes)

In [220]:
gatsby_vector = vectorizer.transform(Mayo_sent_token)
model.predict(Mayo_sent_token)

TypeError: Cannot cast array data from dtype('float64') to dtype('S32') according to the rule 'safe'

In [226]:
#example from sklearn documentation
X = np.random.randint(5, size=(8, 100))
print X
y = np.array([1, 2, 3, 4, 5, 6, 7, 8])
clf = MultinomialNB()
clf.fit(X, y)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
print(clf.predict(X[2:6]))


[[2 0 0 1 0 3 1 0 1 2 1 4 4 0 0 2 2 4 2 1 1 3 0 4 0 3 1 0 4 2 0 2 2 3 3 3 4
  2 2 1 3 3 3 1 4 1 0 3 3 0 3 2 2 4 4 0 4 0 4 0 4 1 4 0 4 3 2 0 0 0 0 1 3 1
  0 0 3 3 0 4 4 0 3 4 1 0 0 1 4 0 1 4 0 2 2 2 4 3 4 4]
 [4 0 2 0 3 2 4 3 1 2 4 1 1 4 1 2 2 2 4 4 4 0 4 2 4 1 2 2 0 3 3 0 0 3 4 2 1
  3 4 2 3 3 4 2 1 3 4 1 0 4 0 3 0 3 3 4 0 3 2 3 3 3 2 2 1 4 0 1 0 2 1 2 0 2
  1 3 2 3 0 4 1 0 4 0 0 4 3 3 0 2 3 2 4 0 0 0 0 4 2 4]
 [4 2 2 3 2 4 1 2 4 3 4 4 1 1 2 1 2 3 4 1 4 1 2 4 4 3 1 0 4 3 3 4 4 1 4 2 2
  0 3 0 1 3 2 4 4 1 2 4 1 4 4 1 1 0 1 2 4 2 3 3 2 0 1 0 2 4 3 0 1 1 4 3 4 4
  3 0 4 4 4 1 1 2 1 3 4 4 1 0 1 3 1 3 4 0 1 2 4 3 4 1]
 [0 0 2 4 4 3 2 3 2 3 2 4 4 2 2 0 0 0 2 1 1 4 1 1 2 1 2 2 3 0 2 2 2 3 4 1 1
  2 2 2 3 4 0 1 4 4 0 4 1 0 3 2 0 1 2 1 4 3 1 1 2 1 2 2 2 0 4 2 0 2 1 0 2 2
  0 2 2 3 3 1 2 2 1 0 1 3 0 3 4 0 2 2 2 2 2 4 1 3 4 4]
 [3 1 2 2 1 3 0 0 1 3 3 3 2 1 0 3 2 1 3 1 3 0 0 3 0 0 1 4 1 2 3 2 4 2 3 4 2
  4 3 2 1 1 3 3 2 4 2 1 0 3 3 1 1 0 4 4 2 1 0 4 1 1 3 3 0 1 4 2 0 2 4 1 0 0
  2 0 3 0 2 0 4 3 3 